# Archive buffer

Define the spaces and the cell factory.

In [56]:
from gym import spaces
from go_explore.cells import CellIsObs

obs_space = spaces.Box(0, 10, (2,))
observation_space = spaces.Dict({"observation": obs_space, "goal": obs_space, "cell": obs_space, "goal_cell": obs_space})
action_space = spaces.Box(0, 0, (1,))

cell_factory = CellIsObs(obs_space)


Define the buffer.

In [57]:
from go_explore.archive import ArchiveBuffer

archive = ArchiveBuffer(
    buffer_size=100,
    observation_space=observation_space,
    action_space=action_space,
    cell_factory=cell_factory,
    n_envs=2,
)


Feed the buffer.

In [58]:
import numpy as np

trajectories = np.array(
    [
        [[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6]],
        [[0, 0], [1, 0], [1, 1], [1, 2], [1, 3], [0, 3], [0, 4]],
    ]
)
goals = np.array(
    [
        [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
        [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
    ]
)
cell_trajectory = cell_factory(trajectories)
goal_cell_trajectory = cell_factory(goals)
observations = [
    {
        "observation": trajectories[:, i],
        "goal": goals[:, i],
        "cell": cell_trajectory[:, i],
        "goal_cell": goal_cell_trajectory[:, i],
    }
    for i in range(7)
]


In [59]:
for i in range(6):
    archive.add(
        obs=observations[i],
        next_obs=observations[i + 1],
        action=np.array([[0], [0]]),
        reward=np.array([0, 0]),
        done=np.ones(2) * (i == 5),
        infos=[{}, {}],
    )


Try sampling trajectory method.

In [60]:
archive.sample_trajectory()

(array([[1., 0.],
        [1., 1.],
        [1., 2.],
        [1., 3.]], dtype=float32),
 array([[1., 0.],
        [1., 1.],
        [1., 2.],
        [1., 3.]], dtype=float32))

Here is the set of possible trajectories.

In [61]:
possible_trajectories = [
    [[0, 1]],
    [[0, 1], [0, 2]],
    [[0, 1], [0, 2], [0, 3]],
    [[0, 1], [0, 2], [0, 3], [0, 4]],
    [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5]],
    [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6]],
    [[1, 0]],
    [[1, 0], [1, 1]],
    [[1, 0], [1, 1], [1, 2]],
    [[1, 0], [1, 1], [1, 2], [1, 3]],
    # [[1, 0], [1, 1], [1, 2], [1, 3], [0, 3]],
    # [[1, 0], [1, 1], [1, 2], [1, 3], [0, 3], [0, 4]],
    # [[1, 0], [1, 1], [1, 2], [1, 3], [0, 3], [0, 4], [0, 5]],
]


Check that all possible trajectories are sampled.
Also check that all sampled trajectories are possible.

In [62]:
sampled_trajectories = [archive.sample_trajectory()[0].astype(int).tolist() for _ in range(30)]  # list convinient to compare

assert np.all([trajectory in possible_trajectories for trajectory in sampled_trajectories])
assert np.all([trajectory in sampled_trajectories for trajectory in possible_trajectories])
